In [1]:
import imaplib
import email
from email.header import decode_header
import base64
from bs4 import BeautifulSoup
import re

import config

# Подключаемся к серверу, логинимся

In [113]:
mail_pass = config.mail_pass
username = config.username
imap_server = "imap.mail.ru"
imap = imaplib.IMAP4_SSL(imap_server)
imap.login(username, mail_pass)

('OK', [b'Authentication successful'])

## Выбираем папку

In [114]:
imap.select("INBOX")

('OK', [b'20'])

### Выбираем все письма, получаем порядковые номера

In [15]:
imap.search(None, 'ALL')

('OK', [b'1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20'])

In [9]:
# M is a connected IMAP4 instance...
typ, msgnums = imap.search(None, 'FROM', '"LDJ"')
print(typ, msgnums)

NO [b'[CANNOT] Unsupported search criterion: FROM']


### Выбираем непрочитанные письма, получаем их порядковые номера

In [16]:
imap.search(None, "UNSEEN")

('OK', [b'12 16 19'])

#### получаем UID непрочитанных писем

In [17]:
imap.uid('search', "UNSEEN", "ALL")

('OK', [b'14 24 28'])

# открываем конкретное письмо (в этом примере 19 номер по порядку и UID-28)

In [116]:
res, msg = imap.fetch(b'19', '(RFC822)')  #Для метода search по порядковому номеру письма
res, msg = imap.uid('fetch', b'28', '(RFC822)')  #Для метода uid

### после этой операции письмо в ящике будет отмечено как прочитанное

In [84]:
msg[0][0]

b'19 (UID 28 RFC822 {110401}'

## получаем объект email из байтов

In [117]:
msg = email.message_from_bytes(msg[0][1])
type(msg)

email.message.Message

## собираем информацию о письме

In [46]:
letter_date = email.utils.parsedate_tz(msg["Date"]) # дата получения, приходит в виде строки, дальше надо её парсить в формат datetime
letter_id = msg["Message-ID"] #айди письма
letter_from = msg["Return-path"] # e-mail отправителя

print(type(letter_date), type(letter_id), letter_id, type(letter_from))

<class 'tuple'> <class 'str'> <1662997113.166751447@f221.i.mail.ru> <class 'str'>


In [47]:
msg["Subject"] # тема письма написана кириллицей и закодирована в base64

'=?UTF-8?B?RndkOiDQn9GA0LjQs9C70LDRiNC10L3QuNC1INCyINC90L7QstGL0Lkg0KI=?=\r\n =?UTF-8?B?0LXRhdC90L7Qv9Cw0YDQuiDQsiDRgdGE0LXRgNC1INCy0YvRgdC+0LrQuNGF?=\r\n =?UTF-8?B?INGC0LXRhdC90L7Qu9C+0LPQuNC5IMKr0JjQoi3Qv9Cw0YDQusK7INC40Lw=?=\r\n =?UTF-8?B?0LXQvdC4INCRLtCg0LDQvNC10LXQstCwINC4INCc0LXQttC00YPQvdCw0YA=?=\r\n =?UTF-8?B?0L7QtNC90YvQuSBTdGFydHVwIEh1Yg==?='

In [ ]:
decode_header(msg["Subject"])

[(b'Fwd: \xd0\x9f\xd1\x80\xd0\xb8\xd0\xb3\xd0\xbb\xd0\xb0\xd1\x88\xd0\xb5\xd0\xbd\xd0\xb8\xd0\xb5 \xd0\xb2 \xd0\xbd\xd0\xbe\xd0\xb2\xd1\x8b\xd0\xb9 \xd0\xa2\xd0\xb5\xd1\x85\xd0\xbd\xd0\xbe\xd0\xbf\xd0\xb0\xd1\x80\xd0\xba \xd0\xb2 \xd1\x81\xd1\x84\xd0\xb5\xd1\x80\xd0\xb5 \xd0\xb2\xd1\x8b\xd1\x81\xd0\xbe\xd0\xba\xd0\xb8\xd1\x85 \xd1\x82\xd0\xb5\xd1\x85\xd0\xbd\xd0\xbe\xd0\xbb\xd0\xbe\xd0\xb3\xd0\xb8\xd0\xb9 \xc2\xab\xd0\x98\xd0\xa2-\xd0\xbf\xd0\xb0\xd1\x80\xd0\xba\xc2\xbb \xd0\xb8\xd0\xbc\xd0\xb5\xd0\xbd\xd0\xb8 \xd0\x91.\xd0\xa0\xd0\xb0\xd0\xbc\xd0\xb5\xd0\xb5\xd0\xb2\xd0\xb0 \xd0\xb8 \xd0\x9c\xd0\xb5\xd0\xb6\xd0\xb4\xd1\x83\xd0\xbd\xd0\xb0\xd1\x80\xd0\xbe\xd0\xb4\xd0\xbd\xd1\x8b\xd0\xb9 Startup Hub',
  'utf-8')]

In [ ]:
decode_header(msg["Subject"])[0][0].decode()

In [50]:
# получить тему письма можно и так, но всё равно нужно декодировать
imap.fetch(b'16', "(BODY[HEADER.FIELDS (Subject)])")

('OK',
 [(b'16 (BODY[HEADER.FIELDS (SUBJECT)] {357}',
   b'Subject: =?UTF-8?B?RndkOiDQn9GA0LjQs9C70LDRiNC10L3QuNC1INCyINC90L7QstGL0Lkg0KI=?=\r\n =?UTF-8?B?0LXRhdC90L7Qv9Cw0YDQuiDQsiDRgdGE0LXRgNC1INCy0YvRgdC+0LrQuNGF?=\r\n =?UTF-8?B?INGC0LXRhdC90L7Qu9C+0LPQuNC5IMKr0JjQoi3Qv9Cw0YDQusK7INC40Lw=?=\r\n =?UTF-8?B?0LXQvdC4INCRLtCg0LDQvNC10LXQstCwINC4INCc0LXQttC00YPQvdCw0YA=?=\r\n =?UTF-8?B?0L7QtNC90YvQuSBTdGFydHVwIEh1Yg==?=\r\n\r\n'),
  b')'])

# Получаем полезную нагрузку! Вложения и текст 

In [89]:
# в payload несколько объектов
msg.get_payload()

In [90]:
# можно проверить и так:
msg.is_multipart()

True

In [118]:
# проходимся по частям обычным способом
payload=msg.get_payload()
for part in payload:
    print(part.get_content_type())  

multipart/alternative
application/pdf


In [119]:
# метод walk раскрывает составные части, и показывает что там есть
for part in msg.walk():
    print(part.get_content_type())

multipart/mixed
multipart/alternative
text/plain
text/html
application/pdf


In [120]:
# если стандартно проходится по частям, получается довольно громоздко 
payload=msg.get_payload()
for part in payload:
    print(part.get_content_type())
    if part.is_multipart():
        level=part.get_payload()
        for l_part in level:
            print(l_part.get_content_type())

multipart/alternative
text/plain
text/html
application/pdf


### Получаем текст письма типа text/plain

In [ ]:
for part in msg.walk():
    if part.get_content_maintype() == 'text' and part.get_content_subtype() == 'plain':
        print(base64.b64decode(part.get_payload()).decode())    

## Получаем вложения

In [123]:
for part in payload:
    print(part.get_content_disposition() == 'attachment')

False
True


In [124]:
for part in msg.walk():
    print(part.get_content_disposition() == 'attachment')

False
False
False
False
True


In [212]:
for part in msg.walk():
    if part.get_content_disposition() == 'attachment':
        print(part.get_filename())
        print(base64.b64decode('=0L/QtdGC0LDQvdC6LnBkZg==').decode())
        print(decode_header(part.get_filename())[0][0].decode())
              

=?UTF-8?B?0L/QtdGC0LDQvdC6LnBkZg==?=
петанк.pdf
петанк.pdf


In [ ]:
# а тут можно увидеть и сам файл
for part in msg.walk():
    if part.get_content_disposition() == 'attachment':
        print(part)